In [2]:
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn as nn
from torch.nn.functional import interpolate 
import matplotlib.pyplot as plt
import pydub
from pydub import AudioSegment
from pathlib import Path
import time
# generation imports
from pippi.soundbuffer import SoundBuffer
from pippi import dsp,fx
import param_generation as pg
import _pickle as pickle
from IPython.display import Audio
from feature_extraction.mir_utils import *
###
import torch.utils.data as utils
import torchvision.transforms as transforms
from PIL import Image
###
import scipy.stats as ss
import common_vars as comv
import imp
import librosa
import librosa.display
import sys
import csv


sys.path.append("..")

from feature_extraction import resnet

import helpers
imp.reload(resnet)
imp.reload(helpers)
imp.reload(comv)
imp.reload(pg)
from feature_extraction import pytorch_models as tm
imp.reload(tm)
from helpers import *

# from common_vars import SR
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

drum_types = ['hat', 'kick', 'shake', 'snare']
# save_path="./generated_sample_pack/"
# device = "cpu"
# stack_size=3
# BATCH_SIZE=1
# NUM_BINS=100

# classes=comv.classes
# classes_ranked=comv.classes_ranked
# cDict={v:i for i,v in enumerate(classes)}

# drum_groups=['tom_low','snare','hihat_closed','rim','synth_noise','clap','kick','hihat_open','tom_mid']


# drum_df=pd.DataFrame(columns=["name","fc","cnnlstm","env+freq","consensus","stack_size"])

/home/asalimi/miniconda3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [20]:
# # transformations to the audio (by default, the SR rate of generations is 44100)
# # model is trained on 5512 SR
SCALE_FACTOR = 1/8

# signal = interpolate(torch.tensor(a).reshape([1,1,-1]),scale_factor = SCALE_FACTOR,recompute_scale_factor=False).reshape(1,-1)
# Audio(signal.reshape([-1]),rate=5512,autoplay=True)

In [21]:
import torchaudio
spec = torchaudio.functional.spectrogram

FREQ_BINS=30
TIME_STEPS=10
class specTrans(object):
    def __init__(self,num_mels=50,sr=SR,time_steps=20,amp_to_power=True):
        self.sr = sr
        self.amp_to_power=amp_to_power
        self.num_mels=num_mels
        self.ampP=torchaudio.transforms.AmplitudeToDB(stype='power',top_db=60)
        self.melP=torchaudio.transforms.MelScale(n_mels=self.num_mels, sample_rate=sr,n_stft=None)
        self.hop_step=time_steps-1
    def __call__(self, sample):
        
        wf=sample
        wf=wf.reshape(-1,len(wf))
        sample_length=self.sr

        num_bins=wf[0].shape[0]
        win_length=self.sr//17
        hop_step=self.sr//self.hop_step
        window=torch.tensor([1]*win_length)
        s=spec(wf, 0, window, num_bins, hop_step, win_length,2,normalized=True)
        s=self.melP(s)
        if self.amp_to_power:
            s=self.ampP(s)
        s = s - s.min()
        s = s/s.abs().max()

        freq=s
        freq[torch.isnan(freq)]=0
        freq=freq
        return freq.detach()
spec_tf = specTrans(FREQ_BINS,time_steps=TIME_STEPS,sr=SR//4,amp_to_power=True)

In [153]:
cnet = resnet.resnet18(in_channel=1, out_channel=2).to(device)
checkpoint = torch.load("feature_extraction/models/resnet18/2.973_0.4691_.checkpoint")

In [85]:

# cnet = tm.Conv_Spec_DVN(embed_only=False,device=device).to(device)
# checkpoint = torch.load("feature_extraction/models/1d_conv/0.401_0.0237_.checkpoint")
# cnet.load_state_dict(checkpoint['model_state_dict'])
# cnet.to(device)
# cnet.eval()

cnet_dvd = tm.Conv_Spec_DVD(embed_only=False,device=device).to(device)
checkpoint = torch.load("feature_extraction/models/1d_conv_dvd/0.565_0.3319_.checkpoint")
cnet_dvd.load_state_dict(checkpoint['model_state_dict'])
cnet_dvd.to(device)
cnet_dvd.eval()
pass

In [86]:
# spec_signal = interpolate(torch.tensor(a).reshape([1,1,-1]),scale_factor = 0.25,recompute_scale_factor=False).reshape(1,-1)
# spectrogram = spec_tf(spec_signal[0].float()).to(device)

In [154]:
# make sound and evaluate
def make_and_eval(stack_size = 1):
  
    out,params = StackMaker(stack_size)
    a = memToAud(out)
    try:
        a = helpers.butter_bandpass_filter(a,30,10000,48000,order=2)
    except:
        pass
    # transformations to the audio (by default, the SR rate of generations is 44100)
    signal = interpolate(torch.tensor(a).reshape([1,1,-1]),scale_factor = SCALE_FACTOR,recompute_scale_factor=False).reshape(1,-1).to(device)
    resnet_signal = interpolate(torch.tensor(a).reshape([1,1,-1]),scale_factor = 0.5, recompute_scale_factor=False).to(device)
    spec_signal = interpolate(torch.tensor(a).reshape([1,1,-1]),scale_factor = 0.25,recompute_scale_factor=False).reshape(1,-1)

    spectrogram = spec_tf(spec_signal[0].float()).to(device)
    with torch.no_grad():
        sig = resnet_signal.float()
        ps = cnet(sig)
        ps_dvd = cnet_dvd(signal,spectrogram)
    return a.reshape(-1),signal,ps.cpu().numpy()[0],torch.sigmoid(ps).cpu().numpy()[0],torch.sigmoid(ps_dvd).cpu().numpy()[0]
a,a_downsampled,ps,pss,ps_dvd = make_and_eval()
print(drum_types[np.argmax(ps_dvd)])
Audio(a.reshape([-1]),rate=SR,autoplay=True)

snare


0.125

In [11]:
Audio(a.reshape([-1]),rate=44100,autoplay=True)
a

array([ 0.00000000e+00, -7.26625794e-09, -4.25172522e-08, ...,
       -1.25586299e-26, -1.25973683e-26, -1.26356982e-26])

In [182]:
# loop until found
biggest_p = []
best_sounds = {}
for i in range(1000):
    print(i,end="\r")
    a,a_ds,ps,pss,pss_dvd= make_and_eval(1)
    if pss[0]>pss[1]:
        print(drum_types[np.argmax(pss_dvd)])
        print(pss[0],pss[1])
        break
    biggest_p.append(ps[0])
Audio(a[0:SR//2].reshape([-1]),rate=44100,autoplay=True)

snare
0.4412858 0.4407992


In [13]:
a

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       2.25462401e-40, 2.24936643e-40, 2.24410334e-40])

In [15]:
# getting the best sound out of n tries

best_sounds = {}
best_sounds = dict.fromkeys(drum_types)
for k in drum_types:
    best_sounds[k] = (0,[0])

for i in range(1000):
    print(i,end="\r")
    a,a_ds,ps,pss,pss_dvd= make_and_eval(1)
    if pss[0]>pss[1]:
        t,p = drum_types[np.argmax(pss_dvd)],pss_dvd[np.argmax(pss_dvd)]
        if best_sounds[t][0]<p:
            best_sounds[t] = (p,a)


In [22]:
import numpy
import IPython.display as ipd
from IPython.display import clear_output
import time 
sr = 22050# sample rate
T = 2.0# seconds
t = numpy.linspace(0, T, int(T*sr), endpoint=False)# time variable
x = 0.5*numpy.sin(2*numpy.pi*440*t)
ipd.Audio(x, rate=sr)

for i in range(3):
    x = 0.1*numpy.sin(i*numpy.pi*440*t)
    ipd.set_matplotlib_close(close=True)
    ipd.display(ipd.Audio(x, rate=sr,autoplay=True))
    time.sleep(1)
    clear_output()

In [90]:
#save kit to file
from scipy.io.wavfile import write
from pippi import dsp
write("feature_extraction/kits/kick.wav",SR,best_sounds["kick"][1])

In [23]:
from pydub import AudioSegment

song = AudioSegment.from_wav("feature_extraction/kits/kick.wav")
k = dsp.read("feature_extraction/kits/kick.wav")

In [ ]:
bpm = 250
num_beats = 8

def make_beat(drum_kit):
    loop = np.zeros(num_beats*60*SR//bpm ) # init silent loop
    loop = add_to_loop(loop,"kick",0,drum_kit)
    loop = add_to_loop(loop,"hat",1,drum_kit)
    loop = add_to_loop(loop,"snare",2,drum_kit)
    loop = add_to_loop(loop,"kick",3,drum_kit)
    loop = add_to_loop(loop,"hat",4,drum_kit)
    loop = add_to_loop(loop,"snare",5,drum_kit)
    loop = add_to_loop(loop,"kick",6,drum_kit)
    loop = add_to_loop(loop,"shake",7,drum_kit)
    return loop

def add_to_loop(loop,drum,beat,drum_kit):
        # given a loop, it puts a drum at beat n
        beat_position = SR*beat*60//bpm
        try:
            drum_sound =  drum_kit[drum][1][0:SR//3]
        except:
            drum_sound = np.zeros(SR)
        start,stop = beat_position,min(beat_position+len(drum_sound),len(loop))
        loop[start:stop] += drum_sound[0:stop-start]
        return loop

def play_loop(loop,sr=SR):
        ipd.set_matplotlib_close(close=True)
        ipd.display(ipd.Audio(loop, rate=sr,autoplay=True))
        loop_length = len(loop)/sr
        time.sleep(loop_length)
        clear_output()


def kit_updater(drum_kit,tries=10,):
    # updates a drum kit for n tries
    for i in range(tries):
        a,a_ds,ps,pss,pss_dvd= make_and_eval(np.random.randint(5))
        if pss[0]>pss[1]:
            t,p = drum_types[np.argmax(pss_dvd)],pss_dvd[np.argmax(pss_dvd)]
            drum_kit[t] = (p,a)

def morph_kit():
    kit_updater_thread = threading.Thread(target=kit_updater,args=(drum_kit,10000))
    kit_updater_thread.start()
    while kit_updater_thread.is_alive():
        loop = make_beat(drum_kit)
        ipd.set_matplotlib_close(close=True)
        ipd.display(ipd.Audio(loop, rate=SR,autoplay=True))
        loop_length = len(loop)/SR
        time.sleep(loop_length)
        clear_output()
# loop = make_beat()
# play_loop(loop)
morph_kit()

In [48]:
# # playing a loop while drum kit is updated in seperate thread
drum_kit = {}
drum_kit = dict.fromkeys(drum_types)


def kit_updater(drum_kit,tries=10,):
    for i in range(tries):
        a,a_ds,ps,pss,pss_dvd= make_and_eval(1)
        if pss[0]>pss[1]:
            t,p = drum_types[np.argmax(pss_dvd)],pss_dvd[np.argmax(pss_dvd)]
            drum_kit[t] = (p,a)
        
def check_kit():
    kit_updater_thread = threading.Thread(target=kit_updater,args=(drum_kit,500))
    kit_updater_thread.start()
    while kit_updater_thread.is_alive():
        time.sleep(1)
check_kit()

KeyboardInterrupt: 